In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from matplotlib import pyplot as plt

tf.test.gpu_device_name()

In [ ]:
data_dir = r'data'
os.listdir(data_dir)
# os.listdir(os.path.join(data_dir, 'crazing'))

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                   image_size=(100, 100),
                                                   batch_size=32)
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

Preprocessing

In [ ]:
# SCALE DATA
data = data.map(lambda x, y: (x/255.0, y))

In [ ]:
scaled_interator = data.as_numpy_iterator()

In [ ]:
## SPLIT DATA

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

Building the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') 
])

In [ ]:
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# model.summary()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

Train

In [ ]:
hist = model.fit(train, epochs=15, validation_data=val, callbacks=[tensorboard_callback])

Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

Evaluate Performance

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
prec = Precision()
rec = Recall()
acc = BinaryAccuracy()

In [ ]:
# for batch in test.as_numpy_iterator(): 
#     X, y = batch
#     yhat = model.predict(X)
#     prec.update_state(y, yhat)
#     rec.update_state(y, yhat)
#     acc.update_state(y, yhat)

AttributeError: 'MapDataset' object has no attribute 'shape'